<a href="https://colab.research.google.com/github/RiverTwilight/Awesome-Machine-Learning-Playground/blob/master/Multi_Agent_APF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

The Artificial Potential Field (APF) algorithm is a method for robot path planning that is used to control the robot for avoiding obstacles in front of it
. The APF algorithm considers the robot as a point in potential fields and then combines stretching toward the target and repulsion of obstacles. The final path of the output is the intended path

# Single Agent APF

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Vector2D:
    def __init__(self, x, y):
        self.deltaX = x
        self.deltaY = y
        self.length = np.sqrt(x**2 + y**2)
        self.direction = [0, 0] if self.length == 0 else [x/self.length, y/self.length]

    def __add__(self, other):
        return Vector2D(self.deltaX + other.deltaX, self.deltaY + other.deltaY)

    def __sub__(self, other):
        return Vector2D(self.deltaX - other.deltaX, self.deltaY - other.deltaY)

    def __mul__(self, k):
        return Vector2D(self.deltaX * k, self.deltaY * k)

    def __div__(self, k):
        return Vector2D(self.deltaX / k, self.deltaY / k)

class Basic_APF:
    def __init__(self, _start, _goal, _obstacles, _k_att, _k_rep, _rr, _step_size, _max_iters, _goal_threashold):
        self.start = _start
        self.goal = _goal
        self.obstacles = _obstacles
        self.k_att = _k_att
        self.k_rep = _k_rep
        self.rr = _rr
        self.step_size = _step_size
        self.max_iters = _max_iters
        self.goal_threashold = _goal_threashold
        self.current_pos = _start
        self.iters = 0
        self.is_path_plan_success = False
        self.path = []

    def attractive(self):
        return (self.goal - self.current_pos) * self.k_att

    def repulsion(self):
        rep = Vector2D(0, 0)
        for obs in self.obstacles:
            t_vec = self.current_pos - obs
            if t_vec.length <= self.rr:
                rep = rep + (Vector2D(t_vec.direction[0], t_vec.direction[1]) * self.k_rep) * ((1.0 / t_vec.length - 1.0 / self.rr) / (t_vec.length**2))
        return rep

    def path_plan(self):
        while self.iters < self.max_iters and (self.current_pos - self.goal).length > self.goal_threashold:
            force_rep = self.repulsion()
            force_att = self.attractive()
            force_vec = force_rep + force_att
            self.current_pos = self.current_pos + Vector2D(force_vec.direction[0], force_vec.direction[1]) * self.step_size
            self.iters += 1
            self.path.append(self.current_pos)
        if (self.current_pos - self.goal).length <= self.goal_threashold:
            self.is_path_plan_success = True
        return self.is_path_plan_success

class Improved_APF(Basic_APF):
    def __init__(self, _start, _goal, _obstacles, _k_att, _k_rep, _rr, _step_size, _max_iters, _goal_threashold):
        super().__init__(_start, _goal, _obstacles, _k_att, _k_rep, _rr, _step_size, _max_iters, _goal_threashold)

    def repulsion(self):
        rep = Vector2D(0, 0)
        n = 2  # Exponent in the repulsion term
        for obs in self.obstacles:
            obs_to_rob = self.current_pos - obs
            rob_to_goal = self.goal - self.current_pos
            if obs_to_rob.length < self.rr:
                rep_1_magnitude = self.k_rep * (1.0 / obs_to_rob.length - 1.0 / self.rr) / (obs_to_rob.length**2) * (rob_to_goal.length**n)
                rep_1 = Vector2D(obs_to_rob.direction[0] * rep_1_magnitude, obs_to_rob.direction[1] * rep_1_magnitude)
                rep_2_magnitude = self.k_rep * (n / 2) * ((1.0 / obs_to_rob.length - 1.0 / self.rr)**2) * (rob_to_goal.length**(n - 1))
                rep_2 = Vector2D(rob_to_goal.direction[0] * rep_2_magnitude, rob_to_goal.direction[1] * rep_2_magnitude)
                rep = rep + (rep_1 + rep_2)
        return rep

In [ ]:
k_att = 1.0
k_rep = 0.8
rr = 2
step_size = 0.2
max_iters = 500
goal_threashold = 0.2
step_size_ = 2
start = Vector2D(0, 0)
goal = Vector2D(15, 15)
obstacles = [Vector2D(2, 3), Vector2D(2, 4), Vector2D(3, 5), Vector2D(5.5, 1), Vector2D(6, 6), Vector2D(10, 6), Vector2D(13, 12), Vector2D(14, 14)]

apf = Improved_APF(start, goal, obstacles, k_att, k_rep, rr, step_size, max_iters, goal_threashold)
is_successed = apf.path_plan()

# Extract the planned path
planned_path = apf.path
print(is_successed)

True


In [ ]:
plt.figure()
plt.plot(start.deltaX, start.deltaY, 'go', label='Start')
plt.plot(goal.deltaX, goal.deltaY, 'ro', label='Goal')
for obs in obstacles:
    plt.plot(obs.deltaX, obs.deltaY, 'ko', label='Obstacle')
x_path = [pos.deltaX for pos in planned_path]
y_path = [pos.deltaY for pos in planned_path]
plt.plot(x_path, y_path, label='Planned path')
plt.legend()
plt.show()